In [6]:
import numpy as np
from scipy.stats import norm

In [4]:
r = 0.008
sigma = 0.2
s_t = 100
T = 30
K = 100
T_0 = 0
t = 0

In [5]:
d1 = ((T-T_0) * (np.log(s_t/K)) + ((r - 0.5 * sigma**2) * (T-t)**2) / 2 + (sigma**2 * (T-t)**3)/(3*(T-T_0)))/(sigma * np.sqrt((T-t)**3/3))

d2 = ((T-T_0) * (np.log(s_t/K)) + ((r - 0.5 * sigma**2) * (T-t)**2) / 2)/(sigma * np.sqrt((T-t)**3/3))

In [7]:
C_G = np.exp(-r*(T-t)) * (s_t * np.exp((r-0.5*sigma**2)*(T-T_0)*((T-t)**2)/2 + (sigma**2*(T-t)**3)/(6*(T-T_0)**2))*norm.cdf(d1) - K*norm.cdf(d2))

In [9]:
C_G

-30.51906811823487

In [10]:
P_G = np.exp(-r*(T-t)) * (K*norm.cdf(-d2) - s_t * np.exp((r-0.5*sigma**2)*((T-t)**2)/(2*(T-T_0)) + (sigma**2*(T-t)**3)/(6*(T-T_0)**2))*norm.cdf(-d1))

In [11]:
P_G

18.933942549168304